In [1]:
#import necessary library.
import tensorflow as tf
import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
#load house price dataset.
data = fetch_california_housing()
x_train_full, x_test, y_train_full, y_test = train_test_split(data.data, data.target)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full) 

In [3]:
#scalling the dataset.
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_valid = sc.transform(x_valid)
x_test = sc.transform(x_test)

In [5]:
!pip install -U keras-tuner

     |████████████████████████████████| 71kB 3.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=c5ac0a4e946a00b9799912572212c4b1385175258a86d2c4a5568bc25468bd54
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=2feccd7027f9ee3c97cface82395e46be714c06dbb805dc28b0c2dca27f56454
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [6]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [7]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [8]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Housing price')

In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [10]:
tuner.search(x_train, y_train,
             epochs=5,
             validation_data=(x_test, y_test))

Trial 5 Complete [00h 01m 06s]
val_mean_absolute_error: 0.37953675786654156

Best val_mean_absolute_error So Far: 0.37385426958401996
Total elapsed time: 00h 04m 55s
INFO:tensorflow:Oracle triggered exit


In [11]:
models = tuner.get_best_models(num_models=2)

In [12]:
tuner.results_summary()

Results summary
Results in project/Housing price
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 7
units_0: 32
units_1: 416
learning_rate: 0.001
units_2: 480
units_3: 352
units_4: 96
units_5: 384
units_6: 352
units_7: 64
units_8: 384
units_9: 32
units_10: 352
units_11: 288
units_12: 128
units_13: 416
units_14: 192
units_15: 416
units_16: 384
units_17: 160
units_18: 224
Score: 0.37385426958401996
Trial summary
Hyperparameters:
num_layers: 8
units_0: 224
units_1: 480
learning_rate: 0.001
units_2: 96
units_3: 160
units_4: 352
units_5: 384
units_6: 512
units_7: 32
units_8: 192
units_9: 352
units_10: 96
units_11: 192
units_12: 256
units_13: 192
units_14: 448
units_15: 320
units_16: 352
units_17: 384
units_18: 288
Score: 0.37953675786654156
Trial summary
Hyperparameters:
num_layers: 15
units_0: 256
units_1: 64
learning_rate: 0.001
units_2: 288
units_3: 128
units_4: 32
units_5: 320
units_6: 352
units_7: 320
units_8: 